In [1]:
import pandas as pd
import numpy as np
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import category_encoders as ce
import pickle
from sklearn.svm import SVC
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
train = pd.read_csv("../data/sample_train.txt", delimiter="\t")
test = pd.read_csv("../data/test_id.txt", delimiter="\t")

In [3]:
df = pd.concat([train, test], axis=0)

In [4]:
risk = pd.read_csv("../features/risk/risk.csv")
df = df.merge(risk, on="id", how="left")

In [5]:
symbol = pd.read_csv("../features/symbol/symbol.csv")
df = df.merge(symbol, on="id", how="left")

In [6]:
# dp_emb = pd.read_csv("../features/graph/graph_filtered_64.emb")
# df = df.merge(dp_emb, on="id", how="left")

In [7]:
dp_emb = pd.read_csv("../features/graph/deepwalk_128_filtered.emb")
df = df.merge(dp_emb, on="id", how="left")

In [8]:
dp_emb.shape

(28442, 129)

In [9]:
# nv_emb = pd.read_csv("../data/edge/node2vec_64.emb")
# df = df.merge(nv_emb, on="id", how="left")

In [10]:
graph_info = pd.read_csv("../features/graph/graph_info.csv")
df = df.merge(graph_info, on="id", how="left")

In [11]:
# app_pca = pd.read_csv("../data/app/app_pca_16.csv")
# df = df.merge(app_pca, on="id", how="left")

app_lda = pd.read_csv("../features/app/app_lda_16.csv")
df = df.merge(app_lda, on="id", how="left")

# app_lda = pd.read_csv("../data1/app/app_lda_16.csv")
# df = df.merge(app_lda, on="id", how="left")

# app_nmf = pd.read_csv("../data/app/app_nmf_16.csv")
# df = df.merge(app_nmf, on="id", how="left")

In [12]:
# app_info = pd.read_csv("../features/app/app_info.csv")
# df = df.merge(app_info, on="id", how="left")

app_graph

In [13]:
# to_pca = pd.read_csv("../features/app_graph/to_app_pca_mean.csv")
# from_pca = pd.read_csv("../features/app_graph/from_app_pca_mean.csv")

# df = df.merge(to_pca, on="id", how="left")
# df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_num_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_num_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_weight_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_weight_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

In [14]:
# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_num_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_num_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_weight_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_weight_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

In [15]:
# to_lda = pd.read_csv("../features/app_graph/to_app_lda_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_num_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_num_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_weight_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_weight_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

risk_graph

In [16]:
# to_risk = pd.read_csv("../features/risk_graph/to_risk_mean.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_mean.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

# to_risk = pd.read_csv("../features/risk_graph/to_risk_sum.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_sum.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

symbol_graph

In [17]:
# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_sum.csv")
# from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_sum.csv")

# df = df.merge(to_cat_weight, on="id", how="left")
# df = df.merge(from_cat_weight, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_mean.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_mean.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_mean.csv")
from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_mean.csv")

df = df.merge(to_cat_num, on="id", how="left")
df = df.merge(from_cat_num, on="id", how="left")

to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_mean.csv")
from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_mean.csv")

df = df.merge(to_cat_weight, on="id", how="left")
df = df.merge(from_cat_weight, on="id", how="left")

knn

In [18]:
# dp_knn = pd.read_csv("../features/knn/dp_knn_3.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_10.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_30.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_100.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_300.csv")
# df = df.merge(dp_knn, on="id", how="left")

中心性

In [19]:
pr = pd.read_csv("../features/graph/pagerank.csv")
df = df.merge(pr, on="id", how="left")

In [20]:
for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_pagerank%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

In [21]:
hits = pd.read_csv("../features/graph/hits.csv")
df = df.merge(hits, on="id", how="left")

for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_hits%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

二度联系人

In [22]:
# 联系人个数
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["", "_num", "_weight"]:
            for f in ["sum", "mean"]:
                a = pd.read_csv("../features/graph/%s_d2_%s%s_%s.csv" % (d1, d2, t, f))
                df = df.merge(a, on="id", how="left")

In [23]:
# 二度联系人PageRank值
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["num", "weight"]:
            for f in ["sum", "mean"]:
                a = pd.read_csv("../features/graph/%s_pg_%s_%s_%s_%s_%s.csv" % (d1, d2, t, f, t, f))
                df = df.merge(a, on="id", how="left")

In [24]:
# for d1 in ["from", "to"]:
#     for d2 in ["from", "to"]:
#         for t in ["num", "weight"]:
#             for f in ["sum", "mean"]:
#                 a = pd.read_csv("../features/graph/%s_hits_%s_%s_%s_%s_%s.csv" % (d1, d2, t, f, t, f))
#                 df = df.merge(a, on="id", how="left")

In [25]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

In [26]:
def  process_feature(train_x, valid_x, test_df):
    result = []
    drop_cols = ['id','label']
    for df in [train_x, valid_x, test_df]:
        result.append(df.drop(drop_cols, axis=1))
    return result 

In [34]:
import os
def cv(df, num_folds, param, model_dir, stratified=True, debug=False):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    train_df = df[df.label.notnull()]
    test_df = df[df.label.isnull()]
    
    seed = 178
    if "seed" in param: 
        seed = param["seed"]
        del param['seed']
    
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    oof_preds = np.zeros(train_df.shape[0])
    all_test_preds = []    
    feature_importance_df = pd.DataFrame()
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_df['label'])):
        train_x, train_y = train_df.iloc[train_idx], train_df['label'].iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], train_df['label'].iloc[valid_idx]
        fold_preds = test_df[["id"]]
        
        train_x, valid_x, test = process_feature(train_x, valid_x, test_df)
        if n_fold == 0:
            print(train_x.shape, valid_x.shape, test.shape)
        
        # xgb
        train_data = xgb.DMatrix(train_x, label=train_y)
        valid_data = xgb.DMatrix(valid_x, label=valid_y)
        test_data = xgb.DMatrix(test)
            
        clf=xgb.train(params,
                      train_data,
                      num_boost_round=10000,
                      evals=[(train_data, 'train'), (valid_data, 'eval')],
                      maximize=True,
                      early_stopping_rounds=50,
                      verbose_eval=100)
        valid_preds = clf.predict(valid_data)
        test_preds = clf.predict(test_data)
#         train_data = lgb.Dataset(train_x, label=train_y)
#         validation_data = lgb.Dataset(valid_x, label=valid_y)

#         clf=lgb.train(params,
#                       train_data,
#                       num_boost_round=10000,
#                       valid_sets=[train_data, validation_data],
#                       valid_names=["train", "valid"],
#                       early_stopping_rounds=100,
#                       verbose_eval=100)

#         valid_preds = clf.predict(valid_x, num_iteration=clf.best_iteration)
#         test_preds = clf.predict(test, num_iteration=clf.best_iteration)

        fold_preds['prob'] = test_preds
        fold_preds['fold_id'] = n_fold + 1
        all_test_preds.append(fold_preds)
        
        oof_preds[valid_idx] = valid_preds
        
#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["feature"] = train_x.columns.tolist()
#         fold_importance_df["importance"] = clf.feature_importance()
#         fold_importance_df["fold"] = n_fold + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, valid_preds)))
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    full_auc = roc_auc_score(train_df['label'], oof_preds)
    print('Full AUC score %.6f' % full_auc)    
    if not debug:
        train_df["prob"] = oof_preds
        train_df[['id', 'prob']].to_csv(model_dir + "pred_train.csv", index= False)

        all_test_preds = pd.concat(all_test_preds, axis=0)
        all_test_preds.to_csv(model_dir + "all_test_preds.csv", index=False)
        
        sub = pd.DataFrame()
        sub['id'] = all_test_preds.id.unique()
        sub.set_index("id", inplace=True)
        sub["prob"] = all_test_preds.groupby("id").prob.mean()
        sub.reset_index().to_csv(model_dir + "sub.txt", index=False)
#     display_importances(feature_importance_df)
    return full_auc

In [35]:
params_list = [
 {'colsample_bylevel': 0.8847040577417382, 'colsample_bytree': 0.8115754738184422, 'eval_metric': 'auc', 'learning_rate': 0.02328934816438338, 'max_depth': 12, 'reg_alpha': 9.358919298710106, 'reg_lambda': 549.7115526017773, 'seed': 6949},
 {'colsample_bylevel': 0.7709093524472894, 'colsample_bytree': 0.8128044282493171, 'eval_metric': 'auc', 'learning_rate': 0.022037554243976538, 'max_depth': 12, 'reg_alpha': 7.847660275194494, 'reg_lambda': 949.2864583794493, 'seed': 3692},
 {'colsample_bylevel': 0.6442016127341352, 'colsample_bytree': 0.8182540826041187, 'eval_metric': 'auc', 'learning_rate': 0.024121992089455414, 'max_depth': 10, 'reg_alpha': 26.033206547673238, 'reg_lambda': 503.21151133307285, 'seed': 3544},
 {'colsample_bylevel': 0.8834580354131929, 'colsample_bytree': 0.8654157043105083, 'eval_metric': 'auc', 'learning_rate': 0.019140376507771548, 'max_depth': 29, 'reg_alpha': 8.798306645013295, 'reg_lambda': 537.7107493297367, 'seed': 393},
 {'colsample_bylevel': 0.6859908791090656, 'colsample_bytree': 0.8345962778421004, 'eval_metric': 'auc', 'learning_rate': 0.021181415031397255, 'max_depth': 29, 'reg_alpha': 5.916095308680992, 'reg_lambda': 511.21237869431116, 'seed': 151}
]
result = []
for i, params in enumerate(params_list):
    model_dir = "../model_output/random/xgb_572/%d/" % i
    result.append(cv(df, 5, params, model_dir))

(15166, 572) (3793, 572) (6000, 572)
[12:19:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[0]	train-auc:0.5	eval-auc:0.5
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[12:19:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[12:19:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[12:19:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[12:19:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[12:19:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[12:19:33] /workspace/src/tree/updater_prune.cc:74: tree pr

[12:19:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=12
[12:19:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes, max_depth=10
[12:19:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=12
[12:19:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=9
[12:19:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=11
[12:19:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=11
[12:19:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=11
[12:19:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=8
[1

[12:19:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=12
[12:19:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=11
[12:19:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 166 extra nodes, 0 pruned nodes, max_depth=12
[12:19:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 178 extra nodes, 0 pruned nodes, max_depth=10
[12:19:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 178 extra nodes, 0 pruned nodes, max_depth=10
[12:19:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=10
[12:19:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=12
[12:19:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=11


[12:19:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=9
[12:19:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=9
[12:19:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=10
[200]	train-auc:0.866792	eval-auc:0.67475
[12:19:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=11
[12:19:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=12
[12:19:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=9
[12:19:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 0 pruned nodes, max_depth=12
[12:19:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 ext

[12:20:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=9
[12:20:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 0 pruned nodes, max_depth=10
[12:20:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=10
[12:20:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=12
[12:20:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=9
[12:20:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=11
[12:20:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=10
[12:20:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=10
[1

...

[13:00:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 314 extra nodes, 0 pruned nodes, max_depth=12
[13:00:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 304 extra nodes, 0 pruned nodes, max_depth=19
[13:00:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 286 extra nodes, 0 pruned nodes, max_depth=16
[13:00:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 286 extra nodes, 0 pruned nodes, max_depth=12
[13:00:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 316 extra nodes, 0 pruned nodes, max_depth=13
[13:00:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 288 extra nodes, 0 pruned nodes, max_depth=13
[13:00:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 294 extra nodes, 0 pruned nodes, max_depth=12
[13:00:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 294 extra nodes, 0 pruned nodes, max_depth=14


[13:00:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 280 extra nodes, 0 pruned nodes, max_depth=16
[13:00:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 308 extra nodes, 0 pruned nodes, max_depth=14
[13:00:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 294 extra nodes, 0 pruned nodes, max_depth=15
[13:00:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 290 extra nodes, 0 pruned nodes, max_depth=16
[13:00:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 308 extra nodes, 0 pruned nodes, max_depth=15
[13:00:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 296 extra nodes, 0 pruned nodes, max_depth=13
[13:00:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 270 extra nodes, 0 pruned nodes, max_depth=12
[13:00:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 308 extra nodes, 0 pruned nodes, max_depth=16


[13:00:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 276 extra nodes, 0 pruned nodes, max_depth=15
[13:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 276 extra nodes, 0 pruned nodes, max_depth=12
[13:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 276 extra nodes, 0 pruned nodes, max_depth=11
[13:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 282 extra nodes, 0 pruned nodes, max_depth=12
[13:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 286 extra nodes, 0 pruned nodes, max_depth=13
[13:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 300 extra nodes, 0 pruned nodes, max_depth=13
[13:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 274 extra nodes, 0 pruned nodes, max_depth=13
[13:00:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 272 extra nodes, 0 pruned nodes, max_depth=13


[13:00:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 276 extra nodes, 0 pruned nodes, max_depth=12
[13:00:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 274 extra nodes, 0 pruned nodes, max_depth=15
[13:00:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=14
[13:00:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 268 extra nodes, 0 pruned nodes, max_depth=12
[13:00:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 262 extra nodes, 0 pruned nodes, max_depth=16
[13:00:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 268 extra nodes, 0 pruned nodes, max_depth=16
[13:00:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 264 extra nodes, 0 pruned nodes, max_depth=13
[13:00:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 240 extra nodes, 0 pruned nodes, max_depth=14


[13:01:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 242 extra nodes, 0 pruned nodes, max_depth=14
[13:01:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=12
[13:01:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=14
[13:01:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 266 extra nodes, 0 pruned nodes, max_depth=15
[13:01:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 248 extra nodes, 0 pruned nodes, max_depth=14
[13:01:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 246 extra nodes, 0 pruned nodes, max_depth=13
[13:01:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 260 extra nodes, 0 pruned nodes, max_depth=16
[13:01:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 244 extra nodes, 0 pruned nodes, max_depth=13


In [36]:
result

[0.69917100031739177,
 0.69851797972431173,
 0.6981220818081546,
 0.69796708734772539,
 0.69792485046698194]

In [31]:
# params_list = [
#     {'colsample_bylevel': 0.6284804159795906, 'colsample_bytree': 0.6107308044696937, 'eval_metric': 'auc', 'learning_rate': 0.018255608599494773, 'max_depth': 10, 'reg_alpha': 1.5735582951738696, 'reg_lambda': 524.568572405397},
#     {'colsample_bylevel': 0.7175594155452033, 'colsample_bytree': 0.6642460809868475, 'eval_metric': 'auc', 'learning_rate': 0.020672621352826173, 'max_depth': 9, 'reg_alpha': 9.182424542286707, 'reg_lambda': 494.6152657684175},
#     {'colsample_bylevel': 0.6214135095918236, 'colsample_bytree': 0.6842062945881913, 'eval_metric': 'auc', 'learning_rate': 0.018923994340229598, 'max_depth': 10, 'reg_alpha': 13.210401954734815, 'reg_lambda': 564.4139254883472},
#     {'colsample_bylevel': 0.6886255566284212, 'colsample_bytree': 0.6543176466397964, 'eval_metric': 'auc', 'learning_rate': 0.02062990285846175, 'max_depth': 12, 'reg_alpha': 6.901192987847831, 'reg_lambda': 512.0849426039205},
#     {'colsample_bylevel': 0.6238215955548236, 'colsample_bytree': 0.6071149920552319, 'eval_metric': 'auc', 'learning_rate': 0.019073987958084054, 'max_depth': 10, 'reg_alpha': 0.06454731215242937, 'reg_lambda': 508.76681063354494}
# ]
# result = []
# for i, params in enumerate(params_list):
#     model_dir = "../model_output/xgb_572/%d/" % i
#     result.append(cv(df, 5, params, model_dir))

(15166, 572) (3793, 572) (6000, 572)
[19:30:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=10
[0]	train-auc:0.52189	eval-auc:0.502297
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[19:30:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=10
[19:30:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=10
[19:30:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=10
[19:30:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=10
[19:30:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=10
[19:30:15] /workspace/src/tree/updater

[19:30:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 344 extra nodes, 0 pruned nodes, max_depth=10
[19:30:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 354 extra nodes, 0 pruned nodes, max_depth=10
[19:30:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 390 extra nodes, 0 pruned nodes, max_depth=10
[19:30:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 352 extra nodes, 0 pruned nodes, max_depth=10
[19:30:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 386 extra nodes, 0 pruned nodes, max_depth=10
[19:30:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 382 extra nodes, 0 pruned nodes, max_depth=10
[19:30:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 392 extra nodes, 0 pruned nodes, max_depth=10
[19:30:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 412 extra nodes, 0 pruned nodes, max_depth=10


[19:30:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 586 extra nodes, 0 pruned nodes, max_depth=10
[19:30:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 632 extra nodes, 0 pruned nodes, max_depth=10
[19:30:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 680 extra nodes, 0 pruned nodes, max_depth=10
[19:30:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 610 extra nodes, 0 pruned nodes, max_depth=10
[19:30:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 574 extra nodes, 0 pruned nodes, max_depth=10
[19:30:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 562 extra nodes, 0 pruned nodes, max_depth=10
[19:30:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 586 extra nodes, 0 pruned nodes, max_depth=10
[19:30:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 582 extra nodes, 0 pruned nodes, max_depth=10


[19:30:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 526 extra nodes, 0 pruned nodes, max_depth=10
[19:30:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 694 extra nodes, 0 pruned nodes, max_depth=10
[200]	train-auc:0.908168	eval-auc:0.66213
[19:30:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 676 extra nodes, 0 pruned nodes, max_depth=10
[19:30:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 654 extra nodes, 0 pruned nodes, max_depth=10
[19:30:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 518 extra nodes, 0 pruned nodes, max_depth=10
[19:30:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 668 extra nodes, 0 pruned nodes, max_depth=10
[19:30:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 602 extra nodes, 0 pruned nodes, max_depth=10
[19:30:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 638 

[19:30:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 610 extra nodes, 0 pruned nodes, max_depth=10
[19:30:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 646 extra nodes, 0 pruned nodes, max_depth=10
[19:30:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 582 extra nodes, 0 pruned nodes, max_depth=10
[19:30:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 546 extra nodes, 0 pruned nodes, max_depth=10
[19:30:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 522 extra nodes, 0 pruned nodes, max_depth=10
[19:30:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 662 extra nodes, 0 pruned nodes, max_depth=10
[19:30:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 598 extra nodes, 0 pruned nodes, max_depth=10
[19:30:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 596 extra nodes, 0 pruned nodes, max_depth=10


...

[19:41:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=9
[19:41:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 0 pruned nodes, max_depth=9
[19:41:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 130 extra nodes, 0 pruned nodes, max_depth=9
[19:41:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=9
[19:41:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=9
[19:41:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=9
[19:41:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=9
[19:41:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=9
[19:41:5

[19:41:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=9
[19:41:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 162 extra nodes, 0 pruned nodes, max_depth=9
[19:41:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=9
[19:41:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=9
[19:41:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 0 pruned nodes, max_depth=9
[19:41:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=9
[19:41:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=9
[19:41:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=9
[19:41:5

[19:42:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=9
[19:42:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=9
[19:42:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=9
[19:42:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 180 extra nodes, 0 pruned nodes, max_depth=9
[19:42:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=9
[19:42:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=9
[19:42:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=9
[19:42:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=9
[19:42:0

[19:42:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 162 extra nodes, 0 pruned nodes, max_depth=9
[19:42:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=9
[19:42:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 146 extra nodes, 0 pruned nodes, max_depth=9
[19:42:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 0 pruned nodes, max_depth=9
[19:42:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 0 pruned nodes, max_depth=9
[19:42:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=9
[19:42:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=9
[19:42:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 146 extra nodes, 0 pruned nodes, max_depth=9
[19:42:4

KeyboardInterrupt: 

In [32]:
result

[0.69639841995649232]
